In [1]:
import json

with open("train.json", "r") as file:
    dataset = json.load(file)
file.close()

In [2]:
import os
import requests

s = requests.Session()

api_base = 'https://api.endpoints.anyscale.com/v1'
token = os.getenv("ANYSCALE_API_KEY")
url = f"{api_base}/chat/completions"

def get_completion(query, model='meta-llama/Meta-Llama-3-70B-Instruct'):
    body = {
    "model": model,
    "messages": [{"role": "system", "content": "You are a specialized system focused on sentence classification of court opinion."},
                {"role": "user", "content": query}],
    "temperature": 0.7
    }

    with s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body) as resp:
        output = resp.json()['choices'][0]['message']['content']
        
    return output

In [3]:
import joblib

top_preds = joblib.load('./vecs/top_preds')
true_labels = joblib.load('./vecs/true_labels')
doc_labels = joblib.load('./vecs/doc_label')

len(top_preds), len(true_labels), len(dataset[0]['annotations'][0]['result'])

(91, 91, 91)

In [9]:
from langchain_community.llms import Ollama
llm = Ollama(model='openchat')

In [4]:
for p in top_preds:
    if len(p)!=5:
        print(len(p))

In [5]:
from tqdm import tqdm

In [29]:
def run_preds(data):
    preds = []
    for i in tqdm(range(3, len(data))):
        curr_sent = data[i]['value']['text']

        prompt = f'''RHETORICAL ROLE:
Rhetorical roles in legal writing refer to the distinct functions or purposes that different parts of a document, such as
a legal opinion, serve in conveying information, persuading the reader, and constructing a coherent argument. These
roles encompass various elements like factual background, legal principles, arguments, counter arguments, and
conclusions, each contributing to the document's overall persuasive and informative structure.
Your task is to label each sentence in the document with one of the following predefined rhetorical roles: 
['Preamble', 'Facts', 'Ruling by Lower Court', 'Issues', 'Argument by Petitioner', 'Argument by Respondent', 'Analysis', 'Statute', \
'Precedent Relied', 'Precedent Not Relied', 'Ratio of the decision', 'Ruling by Present Court', 'NONE']
The definition of each rhetorical role is given below:
Preamble: A typical judgement would start with the court name, the details of parties, lawyers and judges' names, Headnotes. This section typically would end with a keyword like (JUDGEMENT or ORDER etc.) Some supreme court cases also have HEADNOTES, ACTS section. They are also part of Preamble.
Facts: This refers to the chronology of events (but not judgement by lower court) that led to filing the case, and how the case evolved over time in the legal system (e.g., First Information Report at a police station, filing an appeal to the Magistrate, etc.) Depositions and proceedings of current court. Summary of lower court proceedings
Ruling by Lower Court: Judgments given by the lower courts (Trial Court, High Court) based on which the present appeal was made (to the Supreme Court or high court). The verdict of the lower Court, Analysis & the ratio behind the judgement by the lower Court is annotated with this label.
Issues: Some judgements mention the key points on which the verdict needs to be delivered. Such Legal Questions Framed by the Court are ISSUES. E.g. “he point emerge for determination is as follow:- (i) Whether on 06.08.2017 the accused persons in furtherance of their common intention intentionally caused the death of the deceased by assaulting him by means of axe ?”
Argument by Petitioner: Arguments by petitioners' lawyers. Precedent cases argued by petitioner lawyers fall under this but when court discusses them later then they belong to either the relied / not relied upon category. E.g. “learned counsel for petitioner argued that …”
Argument by Respondent: Arguments by respondents lawyers. Precedent cases argued by respondent lawyers fall under this but when court discusses them later then they belong to either the relied / not relied upon category. E.g. “learned counsel for the respondent argued that …”
Analysis: Courts discussion on the evidence,facts presented,prior cases and statutes. These are views of the court. Discussions on how the law is applicable or not applicable to current case. Observations(non binding) from court. It is the parent tag for 3 tags: PRE_RLEIED, PRE_NOT_RELIED and STATUTE i.e. Every statement which belong to these 3 tags should also be marked as ANALYSIS. E.g. “Post Mortem Report establishes that .. “ E.g. “In view of the abovementioned findings, it is evident that the ingredients of Section 307 have been made out ….”
Statute : Text in which the court discusses Established laws, which can come from a mixture of sources - Acts , Sections, Articles, Rules, Order, Notices, Notifications, Quotations directly from the bare act, and so on. Statute will have both the tags Analysis + Statute. E.g. “Court had referred to Section 4 of the Code, which reads as under: "4. Trial of offences under the Indian Penal Code and other laws.-- (1) All offences under the Indian Penal Code (45 of 1860) shall be investigated, inquired into, tried, and otherwise dealt with according to the provisions hereinafter contained”
Precedent Relied: Sentences in which the court discusses prior case documents, discussions and decisions which were relied upon by the court for final decisions. So Precedent will have both the tags Analysis + Precedent. E.g. This Court in Jage Ram v. State of Haryana3 held that: "12. For the purpose of conviction under Section 307 IPC, ….. “
Precedent Not Relied: Sentences in which the court discusses prior case documents, discussions and decisions which were not relied upon by the court for final decisions. It could be due to the fact that the situation in that case is not relevant to the current case.. E.g. This Court in Jage Ram v. State of Haryana3 held that: "12. For the purpose of conviction under Section 307 IPC, ….. “
Ratio of the decision: Main Reason given for the application of any legal principle to the legal issue. This is the result of the analysis by the court. This typically appears right before the final decision. This is not the same as “Ratio Decidendi” taught in the Legal Academic curriculum. E.g. “The finding that the sister concern is eligible for more deduction under Section 80HHC of the Act is based on mere surmise and conjectures also does not arise for consideration.”
Ruling by Present Court: Final decision + conclusion + order of the Court following from the natural / logical outcome of the rationale. E.g. “In the result, we do not find any merit in this appeal. The same fails and is hereby dismissed.”
NONE: If a sentence does not belong to any of the above categories. E.g. “We have considered the submissions made by learned counsel for the parties and have perused the record.”
        
The label of current sentence also depends on the previous sentences. So you will be provided with the last 3 sentences and the labels you previously predicted.

'''
        for j in range(i-3, i):
            sent = data[j]['value']['text']
            if i==3:
                label = data[j]['value']['labels'][0]
                preds.append(label)
            else:
                label = preds[j]
            prompt+=f"SENTENCE: {sent.strip()}" + f"\nLABEL: {label}\n"
        
        prompt+="\nThe label of the current sentence also depends on the label of the most similar sentences so you are given top 5 most similar sentences and their corresponding labels:\n"
        for p in top_preds[i]:
            prompt+= f"SENTENCE: {p[0].strip()}" + f"\nLABEL: {p[1]}\n"

        pred_prompt = f'''You are given one sentence from the document your task is to predict the label of that sentence.
You must use the last three sentences and their labels to predict the label of the current sentence.
You must use the most similar sentences and their labels to predict the label of the current sentence.
Now before predicting the output first lets just think step by step process of how you will solve this problem for the following sentence:
{curr_sent.strip()}'''

        prompt += pred_prompt
    
        response = get_completion(prompt).strip()
        
        preds.append(response)
    return preds

In [30]:
# run_preds(dataset[0]['annotations'][0]['result'])

In [41]:
instructions = run_preds(dataset[0]['annotations'][0]['result'])
len(instructions)

100%|██████████| 88/88 [40:27<00:00, 27.59s/it]


91

In [45]:
print(instructions[3])

Let's break down the step-by-step process to predict the label for the given sentence:

**Step 1: Analyze the sentence**
The sentence is: "This appeal coming on for hearing this day, the court delivered the following :- JUDGMENT"

**Step 2: Consider the last three sentences and their labels**
The last three sentences and their labels are:
1. "This Criminal Appeal is filed under Section 374(2) of the Code of Criminal Procedure, 1973 by the advocate for the appellant praying to set aside the order of conviction and sentence in S.C.No.232/2008 on the file of the II Additional Sessions Judge, Gulbarga and acquit the appellant." - Label: PREAMBLE
2. "AND: The State of Karnataka                     .. RESPONDENT (Through Ratkal Police Station) Represented by Additional State Public Prosecutor, Circuit Bench, Gulbarga. (By Shri S.S.Aspalli, Government Pleader)" - Label: PREAMBLE
3. "BEFORE THE HON'BLE MR.JUSTICE ANAND BYRAREDDY CRIMINAL APPEAL NO.3532 OF 2012 BETWEEN:                         

In [48]:
post_cot_prompt = f'''You are given some instructions.
Your task is to analyse the instructions and pick the final label that is predicted from the instructions.
Your final label should be from one of the following predefined rhetorical roles: 
['Preamble', 'Facts', 'Ruling by Lower Court', 'Issues', 'Argument by Petitioner', 'Argument by Respondent', 'Analysis', 'Statute', \
'Precedent Relied', 'Precedent Not Relied', 'Ratio of the decision', 'Ruling by Present Court', 'NONE']
Your output should only be the label and nothing else.
These are the instructions:
{instructions[8]}'''
print(llm.invoke(post_cot_prompt))

 FAC


In [51]:
def post_cot(instructions):
    final_preds = []
    for instruction in tqdm(instructions, total=len(instructions)):
        post_cot_prompt = f'''You are given some instructions.
Your task is to analyse the instructions and pick the final from the instructions.
Your final label should be from one of the following predefined rhetorical roles: 
['Preamble', 'Facts', 'Ruling by Lower Court', 'Issues', 'Argument by Petitioner', 'Argument by Respondent', 'Analysis', 'Statute', \
'Precedent Relied', 'Precedent Not Relied', 'Ratio of the decision', 'Ruling by Present Court', 'NONE']
Only return the prediction and nothing else.
These are the instructions:
{instruction}'''
        final_pred = llm.invoke(post_cot_prompt)
        final_preds.append(final_pred)
    return final_preds

In [52]:
llm_labels = post_cot(instructions)

100%|██████████| 91/91 [01:05<00:00,  1.40it/s]


In [53]:
llm_labels[-10:], true_labels[-10:]

([' ANALYSIS',
  ' ANALYSIS',
  ' ANALYSIS',
  ' ANALYSIS',
  ' Ruling by Present Court',
  ' RULING_BY_PRESENT_COURT',
  ' RULING_BY_PRESENT_COURT',
  ' RULING_BY_PRESENT_COURT',
  ' RULING_BY_PRESENT_COURT',
  ' NONE'],
 ['ANALYSIS',
  'ANALYSIS',
  'RATIO',
  'ANALYSIS',
  'RPC',
  'RPC',
  'RPC',
  'RPC',
  'RPC',
  'NONE'])

In [54]:
label_abbrv = {
    'preamble': 'PREAMBLE',
    "'preamble'": 'PREAMBLE',
    'facts': 'FAC',
    'fac': 'FAC',
    'ruling by lower court': 'RLC',
    'rlc': 'RLC',
    'issues': 'ISSUE',
    'issue': 'ISSUE',
    'argument by petitioner': 'ARG_PETITIONER',
    'argument_by_petitioner': 'ARG_PETITIONER',
    'arg_petitioner': 'ARG_PETITIONER',
    'argument by respondent': 'ARG_RESPONDENT',
    'arg_respondent': 'ARG_RESPONDENT',
    'analysis': 'ANALYSIS',
    'pre_analysis': 'ANALYSIS',
    'statute': 'STA',
    'precedent relied': 'PRE_RELIED',
    'pre_relied': 'PRE_RELIED',
    'precedent not relied': 'PRE_NOT_RELIED',
    'pre_not_relied': 'PRE_NOT_RELIED',
    'ratio': 'Ratio',
    'ratio of the decision': 'Ratio',
    'ratio of decision': 'Ratio',
    'ratio_of_decision': 'Ratio',
    'ruling by present court': 'RPC',
    'ruling_by_present_court': 'RPC',
    'rpc': 'RPC',
    'none': 'NONE'
}
  
abbrv_label = {v: k for k, v in label_abbrv.items()}
abbrv_label

{'PREAMBLE': "'preamble'",
 'FAC': 'fac',
 'RLC': 'rlc',
 'ISSUE': 'issue',
 'ARG_PETITIONER': 'arg_petitioner',
 'ARG_RESPONDENT': 'arg_respondent',
 'ANALYSIS': 'pre_analysis',
 'STA': 'statute',
 'PRE_RELIED': 'pre_relied',
 'PRE_NOT_RELIED': 'pre_not_relied',
 'Ratio': 'ratio_of_decision',
 'RPC': 'rpc',
 'NONE': 'none'}

In [55]:
preds = []
trues = []
for i, label in enumerate(llm_labels):
    try:
        preds.append(label_abbrv[label.lower().strip()])
        trues.append(true_labels[i])
    except:
        # preds.append("none")
        # trues.append(true_labels[i].lower())
        print(i, label.lower())
        print(true_labels[i].lower())

34  analysis (analysis)
arg_petitioner
39  precedent_reli
arg_respondent
40  analysis (analysis)
arg_respondent
46  analysis (analysis)
arg_respondent
48  analysis (analysis)
arg_respondent
49  analysis (analysis)
analysis
51  analysis (analysis)
analysis


In [56]:
preds[:10], trues[:10]

(['PREAMBLE',
  'PREAMBLE',
  'PREAMBLE',
  'PREAMBLE',
  'NONE',
  'FAC',
  'FAC',
  'FAC',
  'FAC',
  'FAC'],
 ['PREAMBLE',
  'PREAMBLE',
  'PREAMBLE',
  'PREAMBLE',
  'NONE',
  'FAC',
  'FAC',
  'FAC',
  'FAC',
  'FAC'])

In [57]:
from sklearn.metrics import classification_report, accuracy_score, f1_score
acc = accuracy_score(trues, preds)
f1 = f1_score(trues, preds, average="macro")

In [60]:
acc, f1

(0.4642857142857143, 0.4611664197871094)

In [59]:
print(classification_report(trues, preds))

                precision    recall  f1-score   support

      ANALYSIS       0.22      1.00      0.36         7
ARG_PETITIONER       1.00      0.50      0.67        10
ARG_RESPONDENT       1.00      0.08      0.14        13
           FAC       0.41      0.92      0.56        12
         ISSUE       0.00      0.00      0.00         0
          NONE       0.67      1.00      0.80         2
      PREAMBLE       1.00      1.00      1.00         4
    PRE_RELIED       1.00      0.12      0.21        26
         RATIO       0.00      0.00      0.00         1
           RLC       0.50      0.25      0.33         4
           RPC       1.00      1.00      1.00         5

      accuracy                           0.46        84
     macro avg       0.62      0.53      0.46        84
  weighted avg       0.81      0.46      0.42        84



/home/silly_ronny/miniconda3/envs/langchain/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/silly_ronny/miniconda3/envs/langchain/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/silly_ronny/miniconda3/envs/langchain/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 